# Le dilemme itéré du prisonnier

Auteur : Philippe Mathieu, [CRISTAL Lab](http://www.cristal.univ-lille.fr), [SMAC team](https://www.cristal.univ-lille.fr/?rubrique27&eid=17), [Lille University](http://www.univ-lille.fr), email : philippe.mathieu@univ-lille.fr

Contributeurs : Louisa Fodil (CRISTAL/SMAC), Céline Petitpré (CRISTAL/SMAC)

Création : 18/01/2018

## Introduction

Le Dilemme du prisonnier (PD) est un jeu **simultané**, **à deux joueurs**, **à somme non nulle**, mis en évidence par Merill Flood & Melvin Dreschler en 1950 pour montrer que l'équilibre de Nash n'est pas toujours idéal. La version itérée du jeu (IPD) permet d'exprimer des stratégies qui se basent sur l'historique de jeu, et donc apprennent du passé. En 1980 Robert Axelrod organise un concours pour la version itérée du jeu dans lequel l'un des participants, Anatol Rappoport met en évidence la célèbre stratégie TFT. Cette version itérée et cette stratégie TFT sont populariséees en 1984 dans le livre de Robert Axelrod "The Evolution of Cooperation". Depuis, des milliers de publications ont été réalisées sur ce sujet, dans tous les domaines !

Ce notebook a pour objectif de montrer comment établir et comparer des stratégies à ce jeu.

On considère ici une classe `Game` permettant de coder un jeu

----
### Game(tab, actions)
- `tab` : la liste des couples de scores
- `actions` : la liste des stratégies possibles

**Methodes**
- `getDominantStrategies(self, strict='True')` qui imprime une liste d'indices des stratégies non dominées et qui renvoie un nouveau Game avec cette nouvelle matrice
- `getNash(self)` qui renvoie une liste d'indices des équilibres de Nash
- `getPareto(self)` qui renvoie une liste d'indices des équilibres de Pareto

In [ ]:
%run Game.py

dip =[(3,3),(0,5),(5,0),(1,1)]   # Dilemme du prisonnier
g = Game(dip,['C','D'])
g.getNash()

# Une stratégie

Une stratégie a pour objectif de décider du coup à jouer. Outre la matrice des gains, les informations disponibles pour une stratégie sont les coups joués par les deux joueurs dans le passé. Les stratégies les plus simples sont bien évidemment les stratégies qui ne prennent pas en compte ce passé, comme les stratégies qui jouent périodiquement la même séquence de coups. Pour assurer un principe d'autonomie de chaque agent, une stratégie est bien sur capable de fournir son prochain coup, mais s'occupe elle-même de stocker ses précédents coups si necessaire.

#### Créons une classe de stratégies de ce type
La classe que nous définissons ici est très simple : des stratégies qui jouent périodiquement les coups, sans réfléchir beaucoup ! Pour l'instant, elle n'est liée à aucun jeu puisque ce type de comportement peut se retrouver dans n'importe quel jeu.

In [ ]:
from abc import abstractmethod
class Strategy():
    def setMemory(self,mem):
        pass
    
    def getAction(self,tick):
        pass
    
    def __copy__(self):
        pass

    def update(self,x,y):
        pass
    

class Periodic(Strategy):
    def __init__(self, sequence, name=None):
        super().__init__()
        self.sequence = sequence.upper()
        self.step = 0
        self.name = "per_"+sequence if (name == None) else name

    def getAction(self,tick):
        return self.sequence[tick % len(self.sequence)]

    def clone(self):
        object = Periodic(self.sequence, self.name)
        return object

    def update(self,x,y):
        pass
    
print("All is ok")

#### Testons notre classe Periodic

In [ ]:
s1 = Periodic("abc")
print(s1.name,end="\t")
for i in range (0,10):
    print(s1.getAction(i), end=' ')
# IL DOIT Y AVOIR 10 COUPS. çA COMMENCE PAR A ET çA FINIT PAR A    

# Un meeting
Un meeting est une rencontre de deux stratégies durant un certain nombre de coups à un jeu fixé. Le score de chacune est la somme des scores obtenus lors de chaque coup, conformément à la matrice du jeu.

In [ ]:
class Meeting :      
    def __init__(self,game,s1,s2,length=1000):
        self.game = game
        self.s1=s1.clone()
        self.s2=s2.clone()
        self.length=length
        self.nb_cooperation_s1 = 0
        self.nb_cooperation_s2 = 0
        
    def reinit(self):
        self.s1_score=0
        self.s2_score=0
        self.s1_rounds=""
        self.s2_rounds=""
    
    def run(self):
        self.reinit()
        for tick in range(0,self.length):
            c1=self.s1.getAction(tick).upper()
            c2=self.s2.getAction(tick).upper()
            if (c1 == "C"):
                self.nb_cooperation_s1 +=1
            if (c2 == "C"):
                self.nb_cooperation_s2 +=1
            self.s1_rounds+=c1
            self.s2_rounds+=c2
            self.s1.update(c1,c2)
            self.s2.update(c2,c1)
            act=self.game.actions
            self.s1_score+=self.game.scores['x'][act.index(c1),act.index(c2)]
            self.s2_score+=self.game.scores['y'][act.index(c1),act.index(c2)]
            
print("All is ok")

Une rencontre entre deux stratégies est triviale : on initialise un jeu, on crée 2 stratégies, et on les passe au Meeting.

In [ ]:
dip =[(3,3),(0,5),(5,0),(1,1)]   # Dilemme du prisonnier
g = Game(dip,['C','D'])
s1=Periodic("CCD")
s2=Periodic("DDC")
m = Meeting(g,s1,s2,10)
m.run()
print(m.s1.name+"\t"+m.s1_rounds+" "+str(m.s1_score))
print(m.s2.name+"\t"+m.s2_rounds+" "+str(m.s2_score))
# ON DOIT AVOIR 15,35
print()
print("Le nombre de coopérations : " )
print (m.s1.name+"\t" + str(m.nb_cooperation_s1))
print (m.s2.name+"\t" + str(m.nb_cooperation_s2))

# Un tournoi

Un tournoi s'applique à un ensemble de stratégies. Il consiste à faire se rencontrer tout couple de stratégies dans un meeting, y compris chaque stratégie contre elle-même. On appelle ce type de tournoi un "round-robin". On remplit ainsi une matrice carrée de scores. Dans un tel tournoi, le score de chaque stratégie est la somme des scores qu'elle a obtenue. La stratégie gagnante est celle qui obtient le score le plus grand.

Un tournoi est maintenant défini à l'aide de 4 paramètres : le jeu sur lequel il s'applique, l'ensemble des stratégies évaluées, la longueur des parties, le nombre de répétitions effectuées


In [ ]:
import pandas as pd
import numpy as np

class Tournament:
    def __init__(self, game, strategies, length=1000, repeat=1):
        self.strategies = strategies
        self.game = game
        self.length=length
        self.repeat=repeat
        size=len(strategies);
        df = pd.DataFrame(np.zeros((size,size+1),dtype=np.int32))
        df.columns, df.index = [s.name for s in self.strategies]+["Total"], [s.name for s in self.strategies]
        self.matrix = df
        df2 = pd.DataFrame(np.zeros((size,size+1),dtype=np.int32))
        df2.columns, df2.index = [s.name for s in self.strategies]+["Total"], [s.name for s in self.strategies]
        self.cooperations = df2

    def run(self):
        for k in range(self.repeat):
            for i in range (0,len(self.strategies)):
                for j in range (i,len(self.strategies)):
                    meet = Meeting(self.game, self.strategies[i], self.strategies[j], self.length)
                    meet.run()
                    self.matrix.at[self.strategies[i].name, self.strategies[j].name] = meet.s1_score
                    self.matrix.at[self.strategies[j].name, self.strategies[i].name] = meet.s2_score
                    self.cooperations.at[self.strategies[i].name, self.strategies[j].name] = meet.nb_cooperation_s1
                    self.cooperations.at[self.strategies[j].name, self.strategies[i].name] = meet.nb_cooperation_s2
        self.matrix["Total"] = self.matrix.sum(axis=1)
        self.matrix.sort_values(by='Total', ascending=False, inplace=True)
        rows = list(self.matrix.index) + ["Total"]
        self.matrix = self.matrix.reindex(columns=rows)
        self.cooperations["Total"] = self.cooperations.sum(axis=1)
        self.cooperations.sort_values(by='Total', ascending=False, inplace=True)
        rows = list(self.cooperations.index) + ["Total"]
        self.cooperations = self.cooperations.reindex(columns=rows)
        
print("All is ok")

#### Faisons un tournoi
Le Dilemme des prisonniers est un jeu **à somme non nulle** : on ne gagne pas la même chose dans toutes les situations. Il est important de noter que dans le tournoi on ne compte pas le nombre de victoire mais le cumul des scores !

In [ ]:
bag = []
bag.append(Periodic('C'))
bag.append(Periodic('D'))
bag.append(Periodic('DDC'))
bag.append(Periodic('CCD'))
t=Tournament(g,bag,10)
t.run()
print("La matrice de scores : ")
print(t.matrix)
print()
# SUR 10 COUPS : [('per_D', 120), ('per_DDC', 102), ('per_CCD', 78), ('per_C', 60)]
print("La matrice de coopérations : ")
print(t.cooperations)

## Générer des ensembles de stratégies

Constituer une "soupe" de stratégies pour tester la performance d'une stratégie donnée peut être considéré comme subjectif. L'idéal est de constituer des soupes indiscutables, par exemple en constituant des ensembles de toutes les stratégies répondant à une contrainte globale.
Nous pouvons par exemple constituer l'ensemble de toutes les périodiques de période 1 (il y en a 2) et/ou de période 2 (il y en a $2^2=4$) et/ou de période 3 (il y en a $2^3=8$), voir toutes ensembles.
(on notera que l'on génère CCC qui correspond bien évidemment à CC ou C)


In [ ]:
import itertools
cards = ['C','D']
periodics = [p for p in itertools.product(cards, repeat=1)]+[p for p in itertools.product(cards, repeat=2)] + [p for p in itertools.product(cards, repeat=3)]
strats = [Periodic(''.join(p)) for p in periodics] # join to transform in strings
print(str(len(strats))+" stratégies générées")
# 14 SONT GENEREES : 2 à un coup, 4 à deux coups, 8 à trois coups

# Les compétitions écologiques

Les tournois donnent un classement interessant, mais insuffisant. Ils n'étudient pas la robustesse d'une stratégie selon le nombre de représentants des adversaires. La compétition écologique répond à ce problème : elle permet de faire varier les populations de chaque stratégie. Le principe d'une compétition écologique est très simple. Initialement, on considère `n` représentants de chacune des `s` stratégies évaluées. Les `n*s` représentants jouent tous les uns contre les autres dans un tournoi. à l'étape 1, les repésentants de chaque stratégie sont obtenus proportionnellement à leur réussite à l'étape précédente. Meilleur vous-êtes, plus vous aurez de descendants. Une compétition écologique oblige donc une stratégie à être robuste aux changement de nombre d'adversaires. Un classement écologique est donc plus "robuste" qu'un classement de tournoi. On compend immédiatement que pour être bien classé, il est préférable de très bien jouer contre ses propres congénères, puisqu'avec un peu de chance, ils seront de plus en plus nombreux.
Une compétition écologique peut alors se représenter sur un graphique temporel, les générations en abscisse et les populations de chaque stratégie en ordonnée. On utilisera *matplotlib* pour calculer ces graphiques.

In [ ]:
import pandas
import copy
import math
import matplotlib.pyplot as plt
%matplotlib inline

class Ecological:
    def __init__(self, game, strategies, length=1000, repeat=1, pop=100):
        self.strategies = strategies
        self.pop = pop
        self.game = game
        self.length = length
        self.generation = 0 #Numéro de la génération actuelle
        self.base = pop*len(strategies)
        self.historic = pandas.DataFrame(columns = [strat.name for strat in strategies])
        self.historic.loc[0] = [pop for x in range (len(strategies))]
        self.extinctions = dict((s.name,math.inf) for s in strategies)
        self.cooperations =  dict((s.name,0) for s in strategies)
        self.listeCooperations = list()
        self.scores = dict((s.name,0) for s in strategies)
        self.tournament = Tournament(self.game, self.strategies,length,repeat)
        self.tournament.run()
        
    def run(self):
        dead = 0
        stab = False
        while ((self.generation < 1000) and (stab==False)):
            parents = list(copy.copy(self.historic.loc[self.generation]))
            for i in range (len(self.strategies)):
                strat=self.strategies[i].name
                if (self.historic.at[self.generation, strat] != 0):
                    score = 0
                    cooperations = 0
                    for j in range(len(self.strategies)): 
                        strat2 = self.strategies[j].name
                        if (self.historic.at[self.generation, strat2] != 0):
                            if i==j:
                                score+=(self.historic.at[self.generation, strat]-1)*self.tournament.matrix.at[strat,strat2]
                                cooperations+=(self.historic.at[self.generation, strat]-1)*self.tournament.cooperations.at[strat,strat2]
                            else:
                                score+=self.historic.at[self.generation, strat2]*self.tournament.matrix.at[strat,strat2]
                                cooperations+=self.historic.at[self.generation, strat2]*self.tournament.cooperations.at[strat,strat2]
                        self.scores[strat] = score
                        self.cooperations[strat] = cooperations
                    
            total = 0
            totalCooperations = 0
            for strat in self.strategies:
                total+=self.scores[strat.name]*self.historic.at[self.generation, strat.name]
                totalCooperations += self.cooperations[strat.name]*self.historic.at[self.generation, strat.name]
            for strat in self.strategies:        
                parent = self.historic.at[self.generation, strat.name]
                if (self.scores[strat.name] != 0):
                    self.historic.at[self.generation+1, strat.name] = math.floor(self.base*parent*self.scores[strat.name]/total)
                elif (self.scores[strat.name] == 0):
                    self.historic.at[self.generation+1, strat.name] = 0
                    dead += 1
                if ((parent!=0) and (self.historic.at[self.generation+1, strat.name] == 0)):
                    self.extinctions[strat.name] = self.generation+1
                elif (self.historic.at[self.generation+1, strat.name] != 0):
                    self.extinctions[strat.name] = self.historic.at[self.generation+1, strat.name]*1000
                if (dead == len(self.strategies) - 1):
                    stab = True
            self.listeCooperations.append(totalCooperations/(self.base*self.length*len(self.strategies)))
            self.generation+=1
            if (parents == list(self.historic.loc[self.generation])):stab = True
        trie = sorted(self.extinctions.items(), key=lambda t:t[1], reverse=True)
        df_trie = pandas.DataFrame()
        for t in trie :
            df_trie[t[0]]=self.historic[t[0]]
        self.historic = df_trie
        return self.historic

    def saveData(self):
        date = datetime.datetime.now()
        self.historic.to_csv(str(date)+'.csv', sep=';', encoding='utf-8')

    def drawPlot(self,nbCourbes=None,nbLegends=None):
        nbCourbes = len(self.strategies) if (nbCourbes==None) else nbCourbes
        nbLegends = len(self.strategies) if (nbLegends==None) else nbLegends
        strat = self.historic.columns.tolist()
        for i in range(nbCourbes):
            plt.plot(self.historic[strat[i]], label=strat[i] if (i<nbLegends) else '_nolegend_')
        plt.legend(bbox_to_anchor=(0, 1), loc=2, borderaxespad=0.)
        plt.ylabel('Population')
        plt.xlabel('Generation')
        plt.show()
        #date = datetime.datetime.now()
        #plt.savefig(str(date)+'.png', dpi=1000)
    
    def drawCooperations(self):
        plt.plot(self.listeCooperations)
        plt.ylabel('Pourcentage de cooperations')
        plt.xlabel('Generation')
        plt.ylim(0, 101)
        plt.show()
      
print("All is ok")

#### Réaliser une compétition écologique avec All_C (qui coopère toujours) et All_D (qui trahit toujours)
Un fois la compétition réalisée, il est possible d'afficher la courbe, l'évolution de la population et l'évolution des coopérations.

In [ ]:
gentille = Periodic("C","All_C")
mechante = Periodic("D","All_D")
eco = Ecological(g, [gentille, mechante])
eco.run()
print("Evolution de la population")
eco.drawPlot()
print("Historique de la population")
print(eco.historic)
print("Evolution des cooperations")
eco.drawCooperations()

In [ ]:
# EXERCICE

# Faire la même chose avec All_C , All_D, Periodic("CDD") et Periodic("CCD")
gentille = Periodic("C","All_C")
mechante = Periodic("D","All_D")
eco = Ecological(g, [gentille, mechante, Periodic('CDD'), Periodic("CCD")])
eco.run()
print("Evolution de la population")
eco.drawPlot()
print("Historique de la population")
print(eco.historic)
print("Evolution des cooperations")
eco.drawCooperations()

# Des stratégies réactives
Les stratégies sont dites "réactives" si leurs actions dépendent des actions passées de l'adversaire. Certaines sont très simples à comprendre. Parmi les plus célèbres, on trouve
- `Tft` (abbréviation de "tit for tat" ou de "donnant donnant" comme on dirait en Français, et qui joue simplement la même chose que l'adversaire au coup précédent
- `Spiteful`(ou Rancinière en Français) qui coopère tant que l'adversaire a coopéré, mais qui ne lui pardonne jamais s'il a trahi une fois 

In [ ]:
class Tft(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "tft"
        self.hisPast=""
        
    def getAction(self,tick):
        return 'C' if (tick==0) else self.hisPast[-1]

    def clone(self):
        return Tft()

    def update(self,my,his):
        self.hisPast+=his
    
    
class Spiteful(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "spiteful"
        self.hisPast=""
        self.myPast=""
        
    def getAction(self,tick):
        if (tick==0):
                return 'C'
        if (self.hisPast[-1]=='D' or self.myPast[-1]=='D') :
            return 'D'
        else :
            return 'C'

    def clone(self):
        return Spiteful()

    def update(self,my,his):
        self.myPast+=my
        self.hisPast+=his

print("All is ok")

#### Comportement de ces stratégies réactives
Vérifions le comportement de ces deux nouvelles stratégies face à `Periodic("CCD")` dans un Meeting.

In [ ]:
m = Meeting(g,Tft(),Periodic("CCD"),10)
m.run()
print(m.s1.name+"\t"+m.s1_rounds+" "+str(m.s1_score))
print(m.s2.name+"\t"+m.s2_rounds+" "+str(m.s2_score))
print("")
m = Meeting(g,Spiteful(),Periodic("CCD"),10)
m.run()
print(m.s1.name+"\t"+m.s1_rounds+" "+str(m.s1_score))
print(m.s2.name+"\t"+m.s2_rounds+" "+str(m.s2_score))


In [ ]:
# Exercice

# Réaliser une compétititon écologique avec 5 stratégies : All_C, All_D, Tft et Periodic("CCD")
 
eco = Ecological(g, [Periodic('C',"All_C"),Periodic('D',"All_D"),Periodic('CCD'),Tft()])
eco.run()
plt.figure(figsize=(10,8))    # pour définir la taille de la figure
eco.drawPlot()
eco.drawCooperations()
# DANS CETTE EXPERIENCE, All_D GAGNE LE TOURNOI, MAIS C'EST TFT QUI GAGNE LA COMPETITION ECOLOGIQUE !

Cette expérience illustre clairement les phénomènes de type proies/prédateurs qui se produisent. `All_D` gagne le tournoi, elle augmente donc en population. Mais son gain se fait principalement au détriment de `All_C`. Si ce dernier périclite, alors `All_D` périclite aussi faute de stratégie à exploiter. En résulte une magnifique inversion de tendance. Gagner un tournoi, n'est pas gagner une compétition écologique.
Si on regarde l'évolution de la coopération, un autre phénomène apparaît : l'émergence d'une coopération généralisée : sans aucun système de régulation, on est passé de 60% de coopération, à moins de 40% de coopération pour finir à 100% de coopération !

Quelques points fondamentaux :
- Il existe une infinité de stratégies
- il n'y a pas de bonne stratégie dans l'absolu. Il n'y a que des stratégies qui se comportent bien "en général". On ne peut pas jouer optimalement contre tout le monde, notamment à cause du premier coup
- `Tft` ne gagne jamais contre personne
- `All_D` ne perd jamais contre personne
- C'est un jeu à somme non nulle : l'important n'est pas de gagner des meetings mais de gagner des points ! Periodic('D') ne perd jamais mais à quel prix ! à faire toujours la guerre, il ne ramène que très peu de points
- Alexrod le disait déjà : pour être bon à ce jeu il faut :
    - ne pas être agressif (ne pas trahir le premier)
    - être réactif
    - savoir pardonner
- C'est le cas de `Tft` qui se comporte très bien en général, mais depuis Rappoport on a trouvé beaucoup mieux !
- Sans aucun système de régulation, il y a la plupart du temps **émergence de la coopération**

In [ ]:
# Accès aux données des DataFrame

# Attention ! avec un dataframe df
# chaque colonne nommée peut etre utilisée comme attribut
# Si on met un seul crochet, c'est une colonne
# donc tournoi.Total et tournoi['Total'] c'est pareil
# 
# si on met des crochets directs c'est df[Col][Lig]
# que l'on peut donc aussi écrire df.Col[lig]
# Si on utilise les fonctions loc, iloc et at, c'est df.iloc[lig,col]
# donc tournoi['Total'][1] est équivalent à tournoi.iloc[1,4]
# Par ailleurs, par défaut le dataframe utilise des entiers comme index
# dans iloc, on met des index ... sauf si on utilise la notation a:b auquel cas ce sont des adresses relatives

tournoi=eco.tournament.matrix
#print("--- La matrice complète du tournoi triée")
#print(tournoi) 
#print("--- Les gagnants du tournoi")
#print(tournoi['Total']) 
#print("--- Les 3 premiers gagnants du tournoi")
#print(tournoi['Total'][0:3])
#print("--- Les gagnants qui ont fait plus de 10000")
#print(tournoi['Total'][tournoi['Total']>10000])


evol=eco.historic
#print("--- L'historique complet trié")
#print(evol)
#print("--- Les populations finales classées")
#print(evol.iloc[eco.generation])
#print(evol.iloc[-1])
#print(evol.tail(1))
#print("--- Les 2 premiers de la compétition")
#print(evol.iloc[-1][0:2]) 
#print("--- Les derniers survivants")
#print(evol.iloc[-1][evol.iloc[-1]>0]) 
#print("--- la ligne quand tft=340 ?")
#evol.loc[evol.tft==340]
#print("--- A quel indice per_C et per_D se croisent ?")
#print(evol.loc[evol.per_C > evol.per_D].loc[evol.per_D!=0])
# Ecrire l'équivalent de select * from evol where ...
#evol.loc[(evol.tft>300) & (evol.per_D>0)]
# depuis pandas0.13  ... s'écrit
#evol.query('tft>300 & per_D>0')

#eco.drawPlot()


#### Deux autres stratégies réactives classiques
cette fois ce sont des stratégies qui se basent sur le coup majoritairement joué par l'adversaire
- `SoftMajority` : Elle joue le coup majoritaire joué par l'adversaire dans le passé. En cas d'égalité elle coopère.
- `HardMajority`
La différence entre `SoftMajority` et `HardMajority` est ténue. Il est à noter que `HardMajority` est spontanément agressive. cette caractéristique fait qu'elle se comportera d'une manière générale nettement moins bien que `SoftMajority`

In [ ]:
# EXERCICE

# Coder SoftMajority qui joue ce qu'a joué son adversaire en majorité. En cas d'égalité, elle joue coopérer

class SoftMajority(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "softmajo"
        self.nbCooperations = 0
        self.nbTrahisons = 0
        
    def getAction(self,tick):
        if (self.nbCooperations >= self.nbTrahisons):
            return 'C'
        else :
            return 'D'

    def clone(self):
        return SoftMajority()

    def update(self,my,his):
        if (his == 'C'):
            self.nbCooperations += 1
        elif (his == 'D'):
            self.nbTrahisons += 1
            
# Coder HardMajority qui joue ce qu'a joué son adversaire en majorité. En cas d'égalité, elle joue trahir
            
class HardMajority(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "hardmajo"
        self.nbCooperations = 0
        self.nbTrahisons = 0
        
    def getAction(self,tick):
        if (self.nbCooperations > self.nbTrahisons):
            return 'C'
        else :
            return 'D'

    def clone(self):
        return HardMajority()

    def update(self,my,his):
        if (his == 'C'):
            self.nbCooperations += 1
        elif (his == 'D'):
            self.nbTrahisons += 1
            
# Coder Gradual. Gradual coopere au premier coup, puis si l'adversaire vient de la trahir pour la nieme fois, 
# entre dans une periode de réétortion (trahisons successives) de n coups suivie de 2 coups de coopération 
# qui joue ce qu'a joué son adversaire en majorité.

class Gradual(Strategy):
    def __init__(self):
        super().__init__()
        self.name = "gradual"
        self.nbTrahisons = 0
        self.punish = 0
        self.calm = 0
    def getAction(self,tick):
        if (tick==0) : return 'C'
        if self.punish > 0 :
            self.punish-=1
            return 'D'
        if self.calm > 0 :
            self.calm-=1
            return 'C'
        if self.hisLast=='D' : 
            self.punish=self.nbTrahisons - 1
            self.calm=2
            return 'D'
        else: return 'C'

    def clone(self):
        return Gradual()

    def update(self,my,his):
        self.hisLast=his
        if (his == 'D'):
            self.nbTrahisons += 1
  
print("All is ok")

In [ ]:
# Vérifiez leur fonctionnement en Meeting l'une contre l'autre sur 20 tours (on doit avoir DCDCDCDCDC pour hard et CDCDCDCDCD pour soft)
m = Meeting(g,HardMajority(),SoftMajority(),20)
m.run()
print(m.s1.name+"\t"+m.s1_rounds+" "+str(m.s1_score))
print(m.s2.name+"\t"+m.s2_rounds+" "+str(m.s2_score))
print()

# Vérifiez Gradual contre Periodic("CD") sur 40 coups
m = Meeting(g,Periodic("CD"),Gradual(),40)
m.run()
print(m.s1.name+"\t"+m.s1_rounds+" "+str(m.s1_score))
print(m.s2.name+"\t"+m.s2_rounds+" "+str(m.s2_score))


## La famille Memory

In [ ]:
class Mem(Strategy):
    def __init__(self, x, y, genome, name=None):
        self.name = name
        self.x = x
        self.y = y
        self.genome = genome
        if (name == None): #Nom par défaut si l'utilisateur ne le définit pas
            self.name = genome
        self.myMoves = [] #contains my x last moves
        self.itsMoves = [] #contains its y last moves

    def clone(self):
        return Mem(self.x, self.y, self.genome, self.name)

    def getAction(self, tick):
        if (tick < max(self.x, self.y)):
            return self.genome[tick]
        cpt = 0
        for i in range(self.x-1,-1,-1):
            cpt*=2
            if (self.myMoves[i] == 'D'):
                cpt+=1
        for i in range(self.y-1,-1,-1):
            cpt*=2
            if (self.itsMoves[i] == 'D'):
                cpt+=1
        cpt += max(self.x, self.y)
        return self.genome[cpt]

    def update(self, myMove, itsMove):
        if (self.x > 0):
            if(len(self.myMoves) == self.x):
                del self.myMoves[0]
            self.myMoves.append(myMove)
        if (self.y > 0):
            if(len(self.itsMoves) == self.y):
                del self.itsMoves[0]
            self.itsMoves.append(itsMove)
            
print("All is ok")

On notera qu'un grand nombre de stratégies très connues se décrivent sous la forme d'une `memory(X,Y)`<br>
Mem(0,0,'C','allc')<br>
Mem(0,0,'D','alld')<br>
Mem(1,0,'cDC','percd')<br>
Mem(1,0,'dDC','perdc')<br>
Mem(0,1,'cCD','tft')<br>
Mem(0,1,'dCD','mistrust')<br>
Mem(1,1,'cCDDD','spiteful')<br>
Mem(1,1,'cCDDC','pavlov')<br>
Mem(0,2,'ccCCCD','tf2t')<br>
Mem(0,2,'ccCDDD','hard_tft')<br>
Mem(1,2,'ccCCCDCDDD','slow_tft')<br>
Mem(1,2,'ccCDCDDCDD','winner12')<br>
Mem(1,2,'','tft_spiteful')<br>
Mem(1,2,'ccCDDDDDDD','spiteful_cc')<br>

In [ ]:
# Exercice :  Petit test d'équivalence : Faire deux tournois avec Gentille, Mechante, Tft, Spiteful, Periodic('CD') et Periodic('DC').
# Le premier en utilisant Periodic, Tft() et Spiteful()
# Le deuxieme en utilisant seulement des Mem(x,y,"",name)
# Vérifier l'équivalence en imprimant la matrice du tournoi

bag1 = [Periodic('C'),Periodic('D'),Tft(),Spiteful(),Periodic('CD'),Periodic('DC')]
t1=Tournament(g,bag1,100)
t1.run()
print(t1.matrix)
print()

bag2 = [Mem(0,0,'C','allc'),Mem(0,0,'D','alld'),Mem(0,1,'cCD','tft'),
        Mem(1,1,'cCDDD','spiteful'),Mem(1,0,'cDC','percd'),Mem(1,0,'dDC','perdc')]
t2=Tournament(g,bag2,100)
t2.run()
print(t2.matrix)

## Les générer toutes
Pour une famille Mem(x,y), le génome est de taille `max(x,y)`pour l'amorce plus `2^(x+y)` pour toutes les situations s du passé sur `x` coups de l'un et `y` coups de l'autre. Il y a donc `2^(max(x,y)+2^(x+y))` strategies à générer. Pour obtenir tous ces éléments il suffit donc de calculer toutes les instanciations possibles de C et D dans le génome, ce qui se fait, encore une fois , avec un produit cartésien.

| famille  | longueur | nombre   |
|         :-:   |     :-:     | :-:    |
| mem(0,1) | 1+2^1 = 3        | 2^3 = 8 |
| mem(1,0) | 1+2^1 = 3        | 2^3 = 8 |
| mem(1,1) | 1+2^2 = 5        | 2^5 = 32 |
| mem(2,0) | 2+2^2 = 6        | 2^6 = 64 |
| mem(1,2) | 2+2^3 = 10       | 2^10 = 1024 |
| mem(2,1) | 2+2^3 = 10       | 2^10 = 1024 |
| mem(2,2) | 2+2^4 = 18       | 2^18 = 262144 |


In [ ]:
def getAllMemory(x,y):
    if (x+y > 4):
        return "Pas calculable"
    len_genome = max(x,y)+2**(x+y)
    permut = [p for p in itertools.product(['C','D'], repeat=len_genome)]
    genomes = [''.join(p) for p in permut]
    return [Mem(x,y,gen) for gen in genomes]


print("Dans Mem(1,1) il y a "+ str(len(getAllMemory(1,1))) + " stratégies")

## La compétition des Mem(1,1)

In [ ]:
bag3=getAllMemory(1,1)
e2=Ecological(g,bag3)
e2.run()
e2.drawPlot(None,4)
evol=e2.historic
print(evol.iloc[-1][evol.iloc[-1]>0])
# Seules 4 survivent : mem11_cCDDD-spite 2126  , mem11_cCDCD-tft 701 , mem11_cCDDC-pavlov 214 , mem11_cCDCC 158

# Evaluation par synthèse de sous-classes

Les compétitions écologiques offrent un outil de mesure assez fiable de la robustesse d'une stratégies, mais encore insuffisant. Il se peut par exemple que certaines stratégies se sacrifient pour d'autres dans un schema *maitre-esclave*. Avoir une synthèse de centaines voir de milliers de compétitions écologiques dans lesquelles on a enlevé certaines stratégies mesure sans doute une meilleure robustesse. L'une des idées les plus simples consiste à calculer les n compétitions possibles que l'on peut faire en enlevant 1 stratégie à un ensemble de n stratégies. On appelle cette technique la technique des sous-classes.
Nous définissions ici 3 fonctions permettant de réaliser ces sous-classes.
- `subclasses(soupe, n)` qui évalue tous les sous ensembles possibles de taille n dans la soupe
- `subclassesWithOneStrat(soupe, n, strat)` qui évalue Strat dans tous les sous ensembles possibles de taille n dans la soupe en ajoutant systématiquement la stratégie strat
- `subclassesRandomWithOneStrat(p, soupe, n, Strat)` qui réalise p competitions de n strategies choisies au hasard dans la soupe dans lesquelles on rajoute systématiquement Strat

Les évaluations réalisées dans ces fonctions sont des compétitions écologiques.

Ces fonctions renvoient à la fin un tableau avec pour chaque stratégie, sa meilleure place, sa pire place, sa moyenne et son écart-type.

In [ ]:
import statistics
import random

def subClasses(soupe, n):
    if (n > len(soupe)):
        print ("les sous-classes doivent être plus petites que la longueur de n")
        return   
    res = pd.DataFrame(np.nan,[s.name for s in soupe], ["Meilleure place","Pire place", "RankAvg", "RankStd"])   
    #res = pd.DataFrame(np.ones((len(soupe),6 ),dtype=np.float32))
    #res.columns, res.index = ["Meilleure place","Competition Meilleure Place", "Pire place","Competiton Pire Place", "RankAvg", "RankStd"],[s.name for s in soupe]
    for s in soupe:
        res.at[s.name, "Meilleure place"] = len(soupe)
    ranks = dict()
    sousEnsembles = list(itertools.combinations(soupe, n))
    for s in sousEnsembles:
        e = Ecological(g, s)
        e.run()
        #print(e.historic)
        classements = e.historic.iloc[e.generation].rank(0, method="min", ascending=False)
        for strat in s : 
            classement = classements[strat.name]
            if (math.isnan(res.at[strat.name, "Meilleure place"]) or classement < res.at[strat.name, "Meilleure place"]):
                res.at[strat.name, "Meilleure place"] = classement
            if (math.isnan(res.at[strat.name, "Pire place"]) or classement > res.at[strat.name, "Pire place"]):
                res.at[strat.name, "Pire place"] = classement  
            if (strat.name in ranks.keys()):
                ranks[strat.name].append(classement)
            if (strat.name not in ranks.keys()):
                ranks[strat.name] = [classement]      
    for strat in soupe : 
        res.at[strat.name, "RankAvg"] = statistics.mean(ranks[strat.name])
        res.at[strat.name, "RankStd"] = statistics.stdev(ranks[strat.name])
    print(res.sort_values(by = ['RankAvg', 'Meilleure place', 'RankStd', 'Pire place'],  ascending = [True, True, True, True ]))
          

def subClassesWithOneStrat(soupe, n, strategy, printAll = False):
    if (n > len(soupe)):
        print ("les sous-classes doivent être plus petites que la longueur de n")
        return     
    res = pd.DataFrame(np.nan,[s.name for s in soupe+[strategy]], ["Meilleure place", "Pire place", "RankAvg", "RankStd"])   
    sousEnsembles = list(itertools.combinations(soupe, n))
    ranks = dict()
    meilleureComp = []
    pireComp = []
    for s in sousEnsembles:  
        e = Ecological(g, list(s) + [strategy])
        e.run()
        classements = e.historic.iloc[e.generation].rank(0, method="min", ascending=False)
        for strat in  list(s) + [strategy] : 
            classement = classements[strat.name]
            if (math.isnan(res.at[strat.name, "Meilleure place"]) or classement < res.at[strat.name, "Meilleure place"]):
                res.at[strat.name, "Meilleure place"] = classement
                if (strat == strategy):
                    meilleureComp = list(s) + [strategy]
            if (math.isnan(res.at[strat.name, "Pire place"]) or classement > res.at[strat.name, "Pire place"]):
                res.at[strat.name, "Pire place"] = classement 
                if (strat == strategy):
                    pireComp = list(s) + [strategy]
            if (strat.name in ranks.keys()):
                ranks[strat.name].append(classement)
            if (strat.name not in ranks.keys()):
                ranks[strat.name] = [classement]
    for s in soupe+[strategy]:
        if (s.name in ranks.keys()):
            res.at[s.name, "RankAvg"] = statistics.mean(ranks[s.name])
            if (len(ranks[s.name]) > 1):
                res.at[s.name, "RankStd"] = statistics.stdev(ranks[s.name])
    if (printAll) :       
        print(res.sort_values(by = ['RankAvg', 'Meilleure place', 'RankStd', 'Pire place'],  ascending = [True, True, True, True ]))
    else : 
        print("Classement de la stratégie : "+strategy.name)
        print(res.loc[strategy.name,:])
    return meilleureComp, pireComp, strategy



def subClassesRandomWithOneStrat(p, soupe, n, strategy, printAll = False ):
    if (n > len(soupe)):
        "Le nombre de stratégies par compétition doit être inférieur ou égal au nombre de stratégies de la soupe initiale"
        return  
    res = pd.DataFrame(np.nan,[s.name for s in soupe+[strategy]], ["Meilleure place","Pire place", "RankAvg", "RankStd"])
    ranks = dict()
    meilleureComp = []
    pireComp = []
    for i in range (0, p) : 
        #print("Competition "+str(i+1)+ "/"+str(p))
        strategies = []
        strategies.append(strategy)
        indice = [i for i in range (0, len(soupe))]
        for i in range (0, n):
            indiceStrat = random.choice(indice)
            indice.remove(indiceStrat)
            strategies.append(soupe[indiceStrat])
        #print("Les stratégies qui jouent sont : ")
        #for s in strategies :
            #print(s.name)
        e = Ecological(g, strategies)
        e.run()
        classements = e.historic.iloc[e.generation].rank(0, method="min", ascending=False)
        for strat in strategies : 
            classement = classements[strat.name]
            if (math.isnan(res.at[strat.name, "Meilleure place"]) or classement < res.at[strat.name, "Meilleure place"]):
                res.at[strat.name, "Meilleure place"] = classement
                if (strat == strategy):
                    meilleureComp = strategies
            if (math.isnan(res.at[strat.name, "Pire place"]) or classement > res.at[strat.name, "Pire place"]):
                res.at[strat.name, "Pire place"] = classement  
                if (strat == strategy):
                    pireComp = strategies
            if (strat.name in ranks.keys()):
                ranks[strat.name].append(classement)
            if (strat.name not in ranks.keys()):
                ranks[strat.name] = [classement]
    for s in soupe+[strategy]:
        if (s.name in ranks.keys()):
            res.at[s.name, "RankAvg"] = statistics.mean(ranks[s.name])
            if (len(ranks[s.name]) > 1):
                res.at[s.name, "RankStd"] = statistics.stdev(ranks[s.name])
    if (printAll) :   
        print(res.sort_values(by = ['RankAvg', 'Meilleure place', 'RankStd', 'Pire place'],  ascending = [True, True, True, True ]))
    else : 
        print("Classement de la stratégie : "+strategy.name)
        print(res.loc[strategy.name,:])
    return meilleureComp, pireComp, strategy

print("All is OK")

#### Un cas simple : toutes les compétitions de 3 stratégies parmi les classiques

In [ ]:
gentille = Periodic('C')
mechante = Periodic('D')
soupe = [gentille, mechante, Tft(), Spiteful(), Gradual(), SoftMajority(), HardMajority()]
subClasses(soupe, 3)

#### Un cas plus volumineux : toutes les stratégies Mem(1,1) avec une stratégie en moins à chaque fois
Sachant qu'il y a 32 mem(1,1) cette opération réalise donc 32 compétitions de 31 stratégie.

In [ ]:
soupe = getAllMemory(1,1)
subClasses(soupe, len(soupe)-1)

#### Test de la stratégie Spiteful avec tous les triplets de classiques

In [ ]:
gentille = Periodic('C')
mechante = Periodic('D')
soupe = [gentille, mechante, Tft(), Gradual(), SoftMajority(), HardMajority()]
res  = subClassesWithOneStrat(soupe, 3, Spiteful())
#Pour afficher le tableau en entier : 
#res = subClassesWithOneStrat(soupe, 3, Spiteful(), True)

Il est à noter que les objets `subClassesWithOneStrat` et `subClassesRandomWithOneStrat` conservent le 
meilleur et le pire des tournois pour la stratégie `strat`
Au moment de l'affichage du classement des sous-classes il est de ce fait possible d'afficher l'ensemble de stratégies qui a été favorable ou défavorable à la stratégie `Strat`.

In [ ]:
meilleureComp, pireComp, strategy = res
print("La meilleure competition pour la stratégie "+strategy.name +" est : ")
for strat in meilleureComp :
    print(strat.name)
 

#### 20 experiences de 10 stratégies prise au hasard dans mem(1,1) contre Gradual()
Pour les `subclassesRandom`, si une stratégie n'a joué qu'une seule fois alors elle n'a pas d'écart-type (NaN); si elle n'a pas joué du tout alors toutes ses valeurs sont à NaN dans le tableau

In [ ]:
soupe = getAllMemory(1,1)
res = subClassesRandomWithOneStrat(20,soupe, 10, Gradual())
#Pour afficher le tableau en entier : 
#subClassesRandomWithOneStrat(20 soupe, 10, Gradual(), True)

On peut d'ailleurs vérifier la compétition la plus "défavorable" à Gradual (2è :-)

In [ ]:
meilleureComp, pireComp, strategy = res

In [ ]:
pireComp.append(Periodic('C'))
soupe = pireComp
e2=Ecological(g,soupe)
e2.run()
e2.drawPlot(None,5)
evol=e2.historic
print(evol.iloc[-1][evol.iloc[-1]>0])

# Bibliographie

- Robert Axelrod. *The Evolution of Cooperation*. (New York: Basic Books, 1984).
- William Poundstone. *Le dilemme du prisonnier*. Cassini
- JP Delahaye et P Mathieu. *Des surprises dans le monde de la coopération*. Pour la Science, numéro spécial "Les mathématiques sociales", pp 58-66, Juillet 1999.
- Philippe Mathieu, Jean-Paul Delahaye. [New Winning Strategies for the Iterated Prisoner's Dilemma](http://jasss.soc.surrey.ac.uk/20/4/12.html). J. Artificial Societies and Social Simulation 20(4) (2017)
- Bruno Beaufils, Jean-Paul Delahaye et Philippe Mathieu. *Our Meeting with Gradual : A good Strategy for the Itareted Prisoner’s Dilemma*. Intern. Conf. on Artificial Life V (ALIFE V), pp. 159- 165, 16-18 mai 1996, Nara (Japon).
- Martin Nowak et K. Sigmund. *TIT for TAT in Heterogeneous Populations*. Nature, vol. 355, n° 16, pp. 250-253, janvier 1992.
- Nowak M., May R., Sigmund K., *L'arithmétique de l'entraide*, Pour la Science No 214, Août 1995, pp. 56-61 